##  Casamento de Fase para Metassuperfícies – Phase Matching for Metasurfaces

---

### **PT - Resumo**
Este notebook demonstra o processo de **casamento de fase** (phase matching) para metassuperfícies holográficas. Dado um mapa de fase alvo (calculado por algoritmos como GS/ASM) e uma biblioteca de nanoestruturas caracterizadas, o algoritmo encontra a combinação ótima de geometrias (L_x, L_y) que melhor reproduz a distribuição de fase desejada para polarizações TE e TM.

O método utiliza **K-D Tree** para busca eficiente de vizinhos mais próximos no espaço de fases, com suporte opcional para filtros de altura (H) quando múltiplas configurações geométricas estão disponíveis.

<details>
<summary><b>Show English</b></summary>

### **EN - Summary**
This notebook demonstrates the **phase matching** process for holographic metasurfaces. Given a target phase map (calculated by algorithms such as GS/ASM) and a library of characterized nanostructures, the algorithm finds the optimal combination of geometries (L_x, L_y) that best reproduces the desired phase distribution for TE and TM polarizations.

The method uses **K-D Tree** for efficient nearest-neighbor search in phase space, with optional support for height filters (H) when multiple geometric configurations are available.

</details>

---

### 1. 🔧 Imports

In [ ]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from datetime import datetime

# Add src to path
sys.path.insert(0, str(Path().resolve().parent.parent / 'src'))

from meta_library import clean_library, phase_matching

# Configurar matplotlib
plt.rcParams['figure.figsize'] = (12, 4)
plt.rcParams['font.size'] = 10

### 2. ⚙️ Parâmetros | Parameters

---

#### **PT - Descrição**
Os parâmetros abaixo controlam o processo de casamento de fase:

| Parâmetro | Significado | Exemplo |
|------------|-------------|---------||
| `library_path` | Caminho para biblioteca de nanoestruturas limpa (CSV/Parquet) | `"library_cleaned.csv"` |
| `target_phase_te_path` | Mapa de fase alvo TE (arquivo .npy ou .npz) | `"phase_te.npy"` |
| `target_phase_tm_path` | Mapa de fase alvo TM (arquivo .npy ou .npz) | `"phase_tm.npy"` |
| `use_height` | Se True, filtra por altura (H) antes do casamento | `False` |
| `target_height` | Altura alvo (μm) quando `use_height=True` | `0.6` |
| `output_dir` | Diretório para salvar resultados | `"results/meta_library/phase_matching/"` |

O algoritmo minimiza o erro quadrático médio (RMS) entre as fases alvo e as fases disponíveis na biblioteca.

<details>
<summary><b>Show English</b></summary>

#### **EN - Description**
The parameters below control the phase matching process:

| Parameter | Meaning | Example |
|------------|----------|---------||
| `library_path` | Path to cleaned nanostructure library (CSV/Parquet) | `"library_cleaned.csv"` |
| `target_phase_te_path` | TE target phase map (.npy or .npz file) | `"phase_te.npy"` |
| `target_phase_tm_path` | TM target phase map (.npy or .npz file) | `"phase_tm.npy"` |
| `use_height` | If True, filter by height (H) before matching | `False` |
| `target_height` | Target height (μm) when `use_height=True` | `0.6` |
| `output_dir` | Directory to save results | `"results/meta_library/phase_matching/"` |

The algorithm minimizes the root mean square (RMS) error between target phases and available phases in the library.

</details>

---

In [ ]:
# Parâmetros de entrada
library_path = "../../data/meta_library/library_cleaned.csv"  # Ajuste conforme necessário
target_phase_te_path = "../../results/holography/phase_te.npy"  # Exemplo
target_phase_tm_path = "../../results/holography/phase_tm.npy"  # Exemplo

# Opções de casamento
use_height = False  # Filtrar por altura?
target_height = 0.6  # μm (usado apenas se use_height=True)
height_tolerance = 0.05  # μm

# Saída
run_id = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
output_dir = Path(f"../../results/meta_library/phase_matching/demo_{run_id}")
output_dir.mkdir(parents=True, exist_ok=True)

print(f"📁 Resultados serão salvos em: {output_dir}")

### 3. 📚 Teoria – Casamento de Fase | Theory – Phase Matching

---

#### **PT - Descrição**
O casamento de fase busca minimizar o erro entre as fases alvos (φ_TE, φ_TM) e as fases disponíveis na biblioteca de nanoestruturas. Para cada pixel (i,j) do mapa de fases alvo, o algoritmo:

1. **Busca no espaço de fases**: Utiliza K-D Tree para encontrar a nanoestrutura com fase mais próxima de (φ_TE_target, φ_TM_target)
2. **Seleção de geometria**: Retorna as dimensões (L_x, L_y) correspondentes
3. **Filtro opcional de altura**: Quando ativado, restringe a busca a nanoestruturas com altura H próxima ao valor alvo

A métrica de erro é o RMS (Root Mean Square):

$$
\text{RMS} = \sqrt{\frac{1}{N} \sum_{i,j} \left[(\phi_{TE}^{target} - \phi_{TE}^{lib})^2 + (\phi_{TM}^{target} - \phi_{TM}^{lib})^2\right]}
$$

**Referência**: Método baseado em técnicas de otimização geométrica para metassuperfícies holográficas (ver Guo et al. e trabalhos relacionados no repositório docs/references).

<details>
<summary><b>Show English</b></summary>

#### **EN - Description**
Phase matching seeks to minimize the error between target phases (φ_TE, φ_TM) and phases available in the nanostructure library. For each pixel (i,j) of the target phase map, the algorithm:

1. **Phase space search**: Uses K-D Tree to find the nanostructure with phase closest to (φ_TE_target, φ_TM_target)
2. **Geometry selection**: Returns the corresponding dimensions (L_x, L_y)
3. **Optional height filter**: When enabled, restricts search to nanostructures with height H close to target value

The error metric is RMS (Root Mean Square):

$$
\text{RMS} = \sqrt{\frac{1}{N} \sum_{i,j} \left[(\phi_{TE}^{target} - \phi_{TE}^{lib})^2 + (\phi_{TM}^{target} - \phi_{TM}^{lib})^2\right]}
$$

**Reference**: Method based on geometric optimization techniques for holographic metasurfaces (see Guo et al. and related works in docs/references repository).

</details>

---

### 4. 📂 Carregamento da Biblioteca | Loading the Library

---

#### **PT - Descrição**
Carregamos a biblioteca de nanoestruturas que já passou pelo processamento (limpeza, cálculo de fases, etc.).

<details>
<summary><b>Show English</b></summary>

#### **EN - Description**
We load the nanostructure library that has already been processed (cleaning, phase calculation, etc.).

</details>

---

In [ ]:
# Carregar biblioteca
df = pd.read_csv(library_path) if library_path.endswith('.csv') else pd.read_parquet(library_path)

print(f"📊 Biblioteca carregada: {len(df)} registros")
print(f"   Colunas disponíveis: {list(df.columns[:10])}...")
print(f"\n   Verificando colunas necessárias:")
required = ['L_x', 'L_y', 'phase_TE', 'phase_TM', 'amp_TE', 'amp_TM']
for col in required:
    status = '✓' if col in df.columns else '✗'
    print(f"   {status} {col}")

# Mostrar amostra
display(df[['L_x', 'L_y', 'H', 'phase_TE', 'phase_TM', 'amp_TE', 'amp_TM']].head())

### 5. 🎯 Carregamento das Fases Alvo | Loading Target Phases

---

#### **PT - Descrição**
Carregamos os mapas de fase TE e TM calculados previamente (por exemplo, usando GS/ASM).

<details>
<summary><b>Show English</b></summary>

#### **EN - Description**
We load the TE and TM phase maps previously calculated (e.g., using GS/ASM).

</details>

---

In [ ]:
# Carregar fases alvo (ou criar exemplo sintético para demonstração)
try:
    target_phase_te = np.load(target_phase_te_path)
    target_phase_tm = np.load(target_phase_tm_path)
    print("✓ Fases alvo carregadas de arquivos")
except FileNotFoundError:
    print("⚠ Arquivos de fase não encontrados, criando exemplo sintético...")
    # Criar mapas de fase sintéticos para demonstração
    size = (100, 100)
    x, y = np.meshgrid(np.linspace(-np.pi, np.pi, size[1]), np.linspace(-np.pi, np.pi, size[0]))
    target_phase_te = np.mod(x**2 + y**2, 2*np.pi)  # Padrão radial
    target_phase_tm = np.mod(x + y, 2*np.pi)  # Padrão linear

print(f"   Shape TE: {target_phase_te.shape}")
print(f"   Shape TM: {target_phase_tm.shape}")
print(f"   Range TE: [{target_phase_te.min():.3f}, {target_phase_te.max():.3f}] rad")
print(f"   Range TM: [{target_phase_tm.min():.3f}, {target_phase_tm.max():.3f}] rad")

# Visualizar
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
im1 = axes[0].imshow(target_phase_te, cmap='twilight', vmin=0, vmax=2*np.pi)
axes[0].set_title('Fase Alvo TE')
axes[0].axis('off')
plt.colorbar(im1, ax=axes[0], label='Fase (rad)')

im2 = axes[1].imshow(target_phase_tm, cmap='twilight', vmin=0, vmax=2*np.pi)
axes[1].set_title('Fase Alvo TM')
axes[1].axis('off')
plt.colorbar(im2, ax=axes[1], label='Fase (rad)')
plt.tight_layout()
plt.savefig(output_dir / 'target_phases.png', dpi=150, bbox_inches='tight')
plt.show()

### 6. 🔍 Execução do Casamento de Fase | Phase Matching Execution

---

#### **PT - Descrição**
Executamos o algoritmo de casamento de fase usando a função `perform_phase_matching` do módulo.

<details>
<summary><b>Show English</b></summary>

#### **EN - Description**
We execute the phase matching algorithm using the `perform_phase_matching` function from the module.

</details>

---

In [ ]:
# Executar casamento de fase
print("🔄 Executando casamento de fase...")
layout_lx, layout_ly, error_map = phase_matching.perform_phase_matching(
    df=df,
    target_phase_tm=target_phase_tm,
    target_phase_te=target_phase_te,
    use_height=use_height,
    target_height=target_height if use_height else None,
    height_col='H'
)

print(f"✓ Casamento concluído!")
print(f"   Layout L_x: {layout_lx.shape}")
print(f"   Layout L_y: {layout_ly.shape}")
print(f"   Mapa de erro: {error_map.shape}")
print(f"   Erro RMS médio: {np.mean(error_map):.4f} rad")
print(f"   Erro RMS máximo: {np.max(error_map):.4f} rad")

### 7. 📊 Visualização dos Resultados | Results Visualization

---

#### **PT - Descrição**
Visualizamos os layouts geométricos encontrados (L_x e L_y) e o mapa de erro do casamento.

<details>
<summary><b>Show English</b></summary>

#### **EN - Description**
We visualize the geometric layouts found (L_x and L_y) and the matching error map.

</details>

---

In [ ]:
# Visualizar layouts
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

im1 = axes[0].imshow(layout_lx, cmap='viridis')
axes[0].set_title('Layout L_x (μm)')
axes[0].axis('off')
plt.colorbar(im1, ax=axes[0], label='L_x (μm)')

im2 = axes[1].imshow(layout_ly, cmap='viridis')
axes[1].set_title('Layout L_y (μm)')
axes[1].axis('off')
plt.colorbar(im2, ax=axes[1], label='L_y (μm)')

im3 = axes[2].imshow(error_map, cmap='hot')
axes[2].set_title('Mapa de Erro RMS (rad)')
axes[2].axis('off')
plt.colorbar(im3, ax=axes[2], label='Erro RMS (rad)')

plt.tight_layout()
plt.savefig(output_dir / 'layouts_and_error.png', dpi=150, bbox_inches='tight')
plt.show()

# Histograma de erros
plt.figure(figsize=(8, 4))
plt.hist(error_map.flatten(), bins=50, edgecolor='black')
plt.xlabel('Erro RMS (rad)')
plt.ylabel('Frequência')
plt.title('Distribuição de Erros de Casamento de Fase')
plt.grid(alpha=0.3)
plt.savefig(output_dir / 'error_histogram.png', dpi=150, bbox_inches='tight')
plt.show()

### 8. 💾 Salvamento dos Resultados | Saving Results

---

#### **PT - Descrição**
Salvamos os layouts e mapas de erro para uso posterior (fabricação, análise, etc.).

<details>
<summary><b>Show English</b></summary>

#### **EN - Description**
We save the layouts and error maps for later use (fabrication, analysis, etc.).

</details>

---

In [ ]:
# Salvar arrays
np.save(output_dir / 'layout_lx.npy', layout_lx)
np.save(output_dir / 'layout_ly.npy', layout_ly)
np.save(output_dir / 'error_map.npy', error_map)

# Salvar CSVs para fabricação
pd.DataFrame(layout_lx).to_csv(output_dir / 'layout_lx.csv', index=False, header=False)
pd.DataFrame(layout_ly).to_csv(output_dir / 'layout_ly.csv', index=False, header=False)

print(f"✓ Resultados salvos em: {output_dir}")
print(f"   - layout_lx.npy, layout_lx.csv")
print(f"   - layout_ly.npy, layout_ly.csv")
print(f"   - error_map.npy")
print(f"   - Figuras: target_phases.png, layouts_and_error.png, error_histogram.png")

### 9. 🔄 Reprodutibilidade | Reproducibility

---

#### **PT - Descrição**
Para reproduzir estes resultados:

1. **Biblioteca**: Use a mesma biblioteca de nanoestruturas (mesmo arquivo CSV/Parquet)
2. **Fases alvo**: Use os mesmos arquivos de fase TE/TM ou o mesmo método de geração
3. **Parâmetros**: Use os mesmos valores de `use_height`, `target_height`, etc.
4. **Versão**: Código testado com Python 3.10+ e dependências do `requirements.txt`

**Comando CLI equivalente**:
```bash
python src/cli/run_phase_matching.py \
  --library ../../data/meta_library/library_cleaned.csv \
  --target-te ../../results/holography/phase_te.npy \
  --target-tm ../../results/holography/phase_tm.npy \
  --out-dir ../../results/meta_library/phase_matching/run_<timestamp> \
  --preview
```

<details>
<summary><b>Show English</b></summary>

#### **EN - Description**
To reproduce these results:

1. **Library**: Use the same nanostructure library (same CSV/Parquet file)
2. **Target phases**: Use the same TE/TM phase files or generation method
3. **Parameters**: Use the same values for `use_height`, `target_height`, etc.
4. **Version**: Code tested with Python 3.10+ and dependencies from `requirements.txt`

**Equivalent CLI command**:
```bash
python src/cli/run_phase_matching.py \
  --library ../../data/meta_library/library_cleaned.csv \
  --target-te ../../results/holography/phase_te.npy \
  --target-tm ../../results/holography/phase_tm.npy \
  --out-dir ../../results/meta_library/phase_matching/run_<timestamp> \
  --preview
```

</details>

---

### 10. 💡 Considerações Finais | Final Considerations

---

#### **PT - Descrição**
Este notebook demonstrou o casamento de fase para metassuperfícies holográficas. Pontos importantes:

- O erro RMS depende da cobertura da biblioteca (quanto maior e mais densa, menor o erro)
- O filtro de altura (`use_height=True`) é útil quando há restrições de fabricação
- Os layouts L_x e L_y podem ser usados diretamente para fabricação (e-beam lithography, etc.)
- Para integração com holografia: use as fases de saída do GS/ASM como entrada aqui

**Próximos passos sugeridos**:
1. Analisar distribuições de L_x e L_y (verificar se estão dentro dos limites de fabricação)
2. Simular resposta óptica do layout final
3. Exportar para formato GDS/CAD para fabricação

<details>
<summary><b>Show English</b></summary>

#### **EN - Description**
This notebook demonstrated phase matching for holographic metasurfaces. Important points:

- RMS error depends on library coverage (larger and denser = lower error)
- Height filter (`use_height=True`) is useful when there are fabrication constraints
- L_x and L_y layouts can be used directly for fabrication (e-beam lithography, etc.)
- For holography integration: use GS/ASM output phases as input here

**Suggested next steps**:
1. Analyze L_x and L_y distributions (check if within fabrication limits)
2. Simulate optical response of final layout
3. Export to GDS/CAD format for fabrication

</details>

---